In [1]:
import pandas as pd
import string
#Spacy trabalha por trás dos panos com redes neurais convolucionais
import spacy
import random
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

### Etapa 2: Carregamento da base de dados

In [2]:
base_dados = pd.read_csv('../Data/base_treinamento.txt', encoding = 'utf-8')

In [3]:
base_dados.shape

(196, 2)

In [4]:
base_dados.head()

,texto,emocao
0,este trabalho é agradável,alegria
1,gosto de ficar no seu aconchego,alegria
2,fiz a adesão ao curso hoje,alegria
3,eu sou admirada por muitos,alegria
4,adoro como você,alegria


In [5]:
base_dados.tail()

,texto,emocao
191,estou chocado e amedrontado com este assassina...,medo
192,é preciso afugenta com ímpeto este medo do inf...,medo
193,políticos usam suas forças para afugentar e am...,medo
194,o objetivo disso e apenas me amedrontar mais,medo
195,isso me apavora,medo


### Função Pré processamento

In [6]:
pontuacoes = string.punctuation
pontuacoes

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [7]:
from spacy.lang.pt.stop_words import STOP_WORDS
stop_words = STOP_WORDS

In [8]:
len(stop_words)

416

In [9]:
pln = spacy.load('pt_core_news_lg')

In [10]:
def preprocessamento(texto):
  texto = texto.lower()
  documento = pln(texto)
  
  lista = []
  for token in documento:
    #lista.append(token.text)
    lista.append(token.lemma_)

  lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in pontuacoes]
  lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])

  return lista

In [11]:
teste = preprocessamento('Estou aPrendendo 1 10 23 processamento de linguagem natural, Curso em Curitiba')
teste

'aprender processamento linguagem natural cursar curitiba'

### Pré-Processamento: Limpeza

In [12]:
base_dados.head(10)

,texto,emocao
0,este trabalho é agradável,alegria
1,gosto de ficar no seu aconchego,alegria
2,fiz a adesão ao curso hoje,alegria
3,eu sou admirada por muitos,alegria
4,adoro como você,alegria
5,adoro seu cabelo macio,alegria
6,adoro a cor dos seus olhos,alegria
7,somo tão amáveis um com o outro,alegria
8,sinto uma grande afeição por ele,alegria
9,quero agradar meus filhos,alegria


In [13]:
base_dados['texto'] = base_dados['texto'].apply(preprocessamento)

In [14]:
base_dados.head(10)

,texto,emocao
0,trabalhar agradável,alegria
1,gostar ficar aconchegar,alegria
2,adesão cursar hoje,alegria
3,admirar,alegria
4,adorar comer,alegria
5,adorar cabelar maciar,alegria
6,adorar cor olho,alegria
7,somar amável outro,alegria
8,sentir umar afeição,alegria
9,querer agradar filho,alegria


### Tratamento da classe

In [15]:
#Lista com uma string e um dicionário
exemplo_base_dados = [["este trabalho é agradável", {"ALEGRIA": True, "MEDO": False}],
                      ["este lugar continua assustador", {"ALEGRIA": False, "MEDO": True}]]

In [16]:
type(exemplo_base_dados)

list

In [17]:
exemplo_base_dados[0]

['este trabalho é agradável', {'ALEGRIA': True, 'MEDO': False}]

In [18]:
exemplo_base_dados[0][0]

'este trabalho é agradável'

In [19]:
exemplo_base_dados[0][1]

{'ALEGRIA': True, 'MEDO': False}

In [20]:
type(exemplo_base_dados[0][1])

dict

In [21]:
#Criando uma lista vazia
base_dados_final = []
#Para cada texto na coluna de base_dados texto...
#para cada emocao na coluna de base_dados emocao...
# se a emocao for alegria, criamos um dicionario..
# com Alegria sendo true, e medo false.
#  já se a emoção for medo
#  alegria é false, e medo recebe true
#   Por final joga-se essa lista de string + dicionário
#   em uma lista chamada de base_dados_final
for texto, emocao in zip(base_dados['texto'], base_dados['emocao']):
  #print(texto, emocao)
  if emocao == 'alegria':
    dic = ({'ALEGRIA': True, 'MEDO': False})
  elif emocao == 'medo':
    dic = ({'ALEGRIA': False, 'MEDO': True})

  base_dados_final.append([texto, dic.copy()])

In [22]:
len(base_dados_final)

196

In [23]:
base_dados_final[0]

['trabalhar agradável', {'ALEGRIA': True, 'MEDO': False}]

In [24]:
base_dados_final[0][0]

'trabalhar agradável'

In [25]:
base_dados_final[0][1]

{'ALEGRIA': True, 'MEDO': False}

In [26]:
type(base_dados_final[0][1])

dict

In [27]:
base_dados_final

[['trabalhar agradável', {'ALEGRIA': True, 'MEDO': False}],
 ['gostar ficar aconchegar', {'ALEGRIA': True, 'MEDO': False}],
 ['adesão cursar hoje', {'ALEGRIA': True, 'MEDO': False}],
 ['admirar', {'ALEGRIA': True, 'MEDO': False}],
 ['adorar comer', {'ALEGRIA': True, 'MEDO': False}],
 ['adorar cabelar maciar', {'ALEGRIA': True, 'MEDO': False}],
 ['adorar cor olho', {'ALEGRIA': True, 'MEDO': False}],
 ['somar amável outro', {'ALEGRIA': True, 'MEDO': False}],
 ['sentir umar afeição', {'ALEGRIA': True, 'MEDO': False}],
 ['querer agradar filho', {'ALEGRIA': True, 'MEDO': False}],
 ['sentir completamente amar', {'ALEGRIA': True, 'MEDO': False}],
 ['amar', {'ALEGRIA': True, 'MEDO': False}],
 ['alívio', {'ALEGRIA': True, 'MEDO': False}],
 ['dor amenizar finalmente', {'ALEGRIA': True, 'MEDO': False}],
 ['achar apaixonar', {'ALEGRIA': True, 'MEDO': False}],
 ['amar maravilhoso', {'ALEGRIA': True, 'MEDO': False}],
 ['sentir animar', {'ALEGRIA': True, 'MEDO': False}],
 ['sentir hoje', {'ALEGRIA': 

### Criando o classificador 

In [50]:
#Cria um modelo em branco usando pt
modelo = spacy.blank('pt')
#Falando o que será trabalhado na base de dados
#textcat = constante, categorização de textos

#nlp.add_pipe("textcat")
#@Language.component
categorias = modelo.add_pipe("textcat")

#adicionando os y
categorias.add_label("ALEGRIA")
categorias.add_label("MEDO")

#Adicionando o pipe na variavel categorias
#nlp.add_pipe(categorias)

#lista vazia que será o histórico do modelo
historico = []

In [51]:
modelo.pipe_labels

{'textcat': ['ALEGRIA', 'MEDO']}